In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

import jax
from jax import numpy as jnp
from jax.config import config
from flax.training import checkpoints
import flax.linen as nn

import nest_asyncio
nest_asyncio.apply()

from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import NearestNDInterpolator
from scipy.interpolate import interp1d

from scipy.optimize import minimize
from scipy.stats import qmc
# Constants
from scipy.constants import Boltzmann, Avogadro
kb = Boltzmann # [J/K] Boltzman's constant
Na = Avogadro  # [mol-1] Avogadro's Number
R = Na * kb    # [J mol-1 K-1] Ideal gas constant

sys.path.append("../../")
from python_helpers.feanneos import HelmholtzModel
from python_helpers.feanneos import helper_solver_funs, helper_jitted_funs
from python_helpers.transport_properties import TransportModel_PVT_Tinv
from python_helpers import helper_get_alpha
from python_helpers import linear_activation
from python_helpers.data_figures import mie_params_of_vle as mie_params_of

PRECISSION = 'float64'
if PRECISSION == 'float64':
    config.update("jax_enable_x64", True)
    type_np = np.float64
    type_jax = jnp.float64
else:
    config.update("jax_enable_x32", True)
    type_np = np.float32
    type_jax = jnp.float32

np.seterr(all="ignore")

{'divide': 'ignore', 'over': 'ignore', 'under': 'ignore', 'invalid': 'ignore'}

In [2]:
#######################################
# type and optimization configuration #
#######################################

of_type = "vle"
# ranges for sigma, eps, lambda_r
l_bounds = [2.5, 100., 9.]
u_bounds = [3.5, 300., 26.]
m_base = 5
params_file = f'optimized_mie_params_{of_type}.csv'

In [3]:
filename = '../../computed_files/phase_equilibria_fluid.xlsx'
excel_file = pd.ExcelFile(filename)

df_info = pd.read_excel(excel_file, sheet_name='info')
df_vle = pd.read_excel(excel_file, sheet_name='vle')
# critical point information interpolation
input_crit_interp = df_info['alpha'].to_numpy()
output_crit_interp = df_info[['rhocad_model', 'Tcad_model', 'Pcad_model']].to_numpy()
crit_interp = interp1d(input_crit_interp, output_crit_interp.T, fill_value='extrapolate')

# Interpolating VLE
input_vle_interp = df_vle[['alpha', 'Tr_vle_model']].to_numpy()
output_vle_interp = df_vle[['P_vle_model', 'rhov_vle_model', 'rhol_vle_model']].to_numpy()
vle_interp = LinearNDInterpolator(input_vle_interp, output_vle_interp)

interpd_dict = {'crit_interp': crit_interp, 'vle_interp': vle_interp}

In [4]:
######################
# Loading FE-ANN EoS #
######################

ckpt_folder = '../../ann_models/feann_eos'

prefix_params = 'FE-ANN-EoS-params_'

###
Tscale = 'Tinv'
seed = 1
factor = 0.05
EPOCHS = 20000
traind_model_folder = f'models_{Tscale}_factor{factor:.2f}_seed{seed}'
ckpt_folder_model = os.path.join(ckpt_folder, traind_model_folder)
ckpt_Tinv = checkpoints.restore_checkpoint(ckpt_dir=ckpt_folder_model, target=None, prefix=prefix_params)
helmholtz_features = list(ckpt_Tinv['features'].values())
helmholtz_model = HelmholtzModel(features=helmholtz_features)
helmholtz_params = {'params': ckpt_Tinv['params']}
fun_dic_fluid = helper_jitted_funs(helmholtz_model, helmholtz_params)


In [5]:
# Reading data from NIST
filename = 'krypton.xlsx'
DataFile = pd.ExcelFile(filename)

if not os.path.exists(params_file):
    sampler = qmc.Sobol(d=3, scramble=False)
    sample = sampler.random_base2(m=m_base)

    scaled_sample = qmc.scale(sample, l_bounds, u_bounds)

    sigma_guess = scaled_sample[:, 0]
    epsilon_guess = scaled_sample[:, 1]
    lr_guess = scaled_sample[:, 2]

    n = len(sigma_guess)

    index = np.arange(n)
    solved = np.zeros(n, dtype=bool)
    sigma_sol = np.nan * np.ones(n)
    epsilon_sol = np.nan * np.ones(n)
    lr_sol = np.nan * np.ones(n)
    of = np.nan * np.ones(n)
    solved_success = np.zeros(n, dtype=bool)

    df = pd.DataFrame({'index': index, 
                    'sigma_guess': sigma_guess, 'epsilon_guess': epsilon_guess, 'lr_guess': lr_guess,
                    'optimized': solved, 
                    'sigma_sol': sigma_sol, 'epsilon_sol': epsilon_sol, 'lr_sol': lr_sol, 'of': of, 'solved_success': solved_success})
    
    df.to_csv(params_file, index=False)


In [6]:
# bounds to control sigma, epsilon and lr
bounds = ((None, None), (None, None), (7., 34.))

kwargs = {'DataFile': DataFile,
          'fun_dic': fun_dic_fluid,
          'interpd_dict': interpd_dict,
          'lambda_a': 6.,
          'weight_rhov_vle':0.0,
          'weight_hvap':1.0,
          'add_critical_point': True,    
          }
args = tuple(kwargs.values())

In [7]:
df = pd.read_csv(params_file)
n = len(df)
for i in range(n):
    if not df.loc[i, 'optimized']:
        inc0 = df.loc[i, ['sigma_guess', 'epsilon_guess', 'lr_guess']].to_list()
        sol = minimize(mie_params_of, inc0, method='Nelder-Mead', args=args, bounds=bounds, options={'maxiter':10})

        df.loc[i, 'solved_success'] = sol.success
        df.loc[i, 'sigma_sol'] = sol.x[0]
        df.loc[i, 'epsilon_sol'] = sol.x[1]
        df.loc[i, 'lr_sol'] = sol.x[2]
        df.loc[i, 'of'] = sol.fun
        df.loc[i, 'optimized'] = True

        df.to_csv(params_file, index=False)
    print(i, df.loc[i, 'optimized'], df.loc[i, 'of'])

0 True 0.6795547137087261
1 True 0.0301131510567224
2 True 0.0031063097187746
3 True 0.1689727418472533
4 True 0.053142377976113
5 True 0.0704009277862588
6 True 3.177944931822786
7 True 0.034586290601701
8 True 0.1417088509706089
9 True 0.0193364337273965
10 True 3.177999709468267
11 True 0.1694953591535302
12 True 0.3701940638823479
13 True 0.0332928003837756
14 True 0.0444057082481074
15 True 0.5164824960391455
16 True 0.3256775247791157
17 True 0.0832735233749559
18 True 0.003106309718795
19 True 0.0310366132543511
20 True 3.1785440068809976
21 True 0.003106309718784
22 True 0.0422992653825502
23 True 0.0729761462439163
24 True 0.4834092350770875
25 True 0.0031063097187819
26 True 0.0091848750213039
27 True 0.0519408765693231
28 True 0.2960137918557918
29 True 0.0202341250268205
30 True 3.1776953043346308
31 True 0.4229536563795009


In [8]:
df = pd.read_csv(params_file)
n = len(df)

of_order = np.argsort(df['of'].to_numpy())

for i in of_order[:5]:
    if not df.loc[i, 'solved_success']:
        inc0 = df.loc[i, ['sigma_sol', 'epsilon_sol', 'lr_sol']].to_list()

        sol = minimize(mie_params_of, inc0, method='Nelder-Mead', args=args, bounds=bounds)

        df.loc[i, 'solved_success'] = sol.success
        df.loc[i, 'sigma_sol'] = sol.x[0]
        df.loc[i, 'epsilon_sol'] = sol.x[1]
        df.loc[i, 'lr_sol'] = sol.x[2]
        df.loc[i, 'of'] = sol.fun
        df.loc[i, 'optimized'] = True

        df.to_csv(params_file, index=False)


In [9]:
of_order = np.argsort(df['of'].to_numpy())
df.loc[of_order].head(n=10)

,index,sigma_guess,epsilon_guess,lr_guess,optimized,sigma_sol,epsilon_sol,lr_sol,of,solved_success
2,2,3.25000,150.00,13.25000,True,3.668625,172.848947,13.367881,0.003106,True
25,25,3.15625,231.25,9.53125,True,3.668625,172.848990,13.367888,0.003106,True
21,21,3.46875,218.75,14.84375,True,3.668625,172.848948,13.367882,0.003106,True
18,18,3.34375,143.75,12.71875,True,3.668625,172.848968,13.367884,0.003106,True
26,26,3.40625,181.25,22.28125,True,3.696148,199.993154,18.721678,0.009185,True
9,9,3.18750,262.50,16.43750,True,3.730556,194.444444,17.046296,0.019336,False
29,29,3.28125,256.25,20.15625,True,3.678241,211.485340,21.475116,0.020234,False
1,1,3.00000,200.00,17.50000,True,3.698148,193.209877,16.398148,0.030113,False
19,19,2.84375,243.75,21.21875,True,3.812731,214.861111,23.222743,0.031037,False
13,13,3.31250,237.50,22.81250,True,3.663421,214.660851,23.756110,0.033293,False


In [10]:
param_best = df.loc[of_order[0], ['sigma_sol', 'epsilon_sol', 'lr_sol']].to_list()
param_best

[3.6686248712377054, 172.84894661230834, 13.367881277569516]